#### Read & Write en Delta Lake

1. Escribir datos en Delta Lake (Managed Table)
2. Escribir datos en Delta Lake (External Table)
3. Leer datos desde Delta Lake (Table)
4. Leer datos desde Delta Lake (File)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS movie_demo
LOCATION "/mnt/sacjccmoviehistory/demo"

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
movie_schema = StructType( fields= [
    StructField("movieId", IntegerType(), False),
    StructField("title", StringType(), True),
    StructField("budget", DoubleType(), True),
    StructField("homePage", StringType(), True),
    StructField("overview", StringType(), True),
    StructField("popularity", DoubleType(), True),
    StructField("yearReleaseDate", IntegerType(), True),
    StructField("releaseDate", DateType(), True),
    StructField("revenue", DoubleType(), True),
    StructField("durationTime", IntegerType(), True),
    StructField("movieStatus", StringType(), True),
    StructField("tagline", StringType(), True),
    StructField("voteAverage", DoubleType(), True),
    StructField("voteCount", IntegerType(), True)
])

In [0]:
movie_df = spark.read \
          .option("header", True) \
          .schema(movie_schema) \
          .csv("/mnt/sacjccmoviehistory/bronze/2024-12-30/movie.csv")

In [0]:
# Crear una tabla administrada por Databricks
movie_df.write.format("delta").mode("overwrite").saveAsTable("movie_demo.movies_managed")

In [0]:
%sql
SELECT * FROM movie_demo.movies_managed;

In [0]:
# Crear una tabla externa
movie_df.write.format("delta").mode("overwrite").save("/mnt/sacjccmoviehistory/demo/movies_external")

In [0]:
%sql
-- Creamos una tabla en base los datos en formatos DELTA, creados en la celda anterior
-- Creacion de una tabla en base a la carpeta movies_extarnal
CREATE TABLE movie_demo.movies_external
USING DELTA
LOCATION "/mnt/sacjccmoviehistory/demo/movies_external"

In [0]:
%sql SELECT * FROM movie_demo.movies_external

In [0]:
# Leer los datos de la carpeta "movies_external" y cargarlos en un DataFrame 
# Se cargar los datos en un DataFrame y no en una tabla.
movies_external_df = spark.read.format("delta").load("/mnt/sacjccmoviehistory/demo/movies_external")

In [0]:
# Particionar los datos de un DataFrame por una columna especifica
# Se creara una tabla administrada en la cual los datos van a estar particionados por la columna yearReleaseDate
movie_df.write.format("delta").mode("overwrite").partitionBy("yearReleaseDate").saveAsTable("movie_demo.movies_partitioned")

In [0]:
%sql
-- Optener los valores de las distintas particiones por dicha columna(yearReleaseDate)
SHOW PARTITIONS movie_demo.movies_partitioned

#### Update & Delete en Delta Lake

1. Update desde Delta Lake
2. Delete desde Delta Lake

- Actualizar y eliminar registros en una tabla Delta Lake
- Delta Lake soporta actualizaciones, eliminaciones y fuciones
- [Documentacion Delta Lake](https://docs.delta.io/latest/delta-update.html)

In [0]:
%sql
SELECT * FROM movie_demo.movies_managed;

- Actualizar valores de la tabla movies_managed (SQL)

In [0]:
%sql
UPDATE movie_demo.movies_managed
SET durationTime = 60
WHERE yearReleaseDate = 2012;

In [0]:
%sql SELECT * FROM movie_demo.movies_managed WHERE yearReleaseDate = 2012;

- Actualizar valores de la tabla movies_managed (Python)

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/mnt/sacjccmoviehistory/demo/movies_managed")

# Declare the predicate by using a SQL-formatted string
deltaTable.update(
    condition="yearReleaseDate = 2013",
    set = {"durationTime":"100"}
)


In [0]:
%sql
SELECT * FROM movie_demo.movies_managed WHERE yearReleaseDate = 2013;

- Eliminar Datos (SQL)

In [0]:
%sql
DELETE FROM movie_demo.movies_managed
WHERE yearReleaseDate = 2014;

In [0]:
%sql
SELECT * FROM movie_demo.movies_managed WHERE yearReleaseDate = 2014;

Eliminar valores (Python)

In [0]:
from delta.tables import *

deltaTable = DeltaTable.forPath(spark, "/mnt/sacjccmoviehistory/demo/movies_managed")

# Declare the predicate by using a SQL-formatted string
deltaTable.delete("yearReleaseDate = 2015")


In [0]:
%sql
SELECT * FROM movie_demo.movies_managed WHERE yearReleaseDate = 2015;

#### Merge/Upsert en Delta Lake

1. Permite insertar los datos de registros que se reciban
2. Actualizar los registros existentes

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, DateType

In [0]:
movie_schema = StructType( fields= [
    StructField("movieId", IntegerType(), False),
    StructField("title", StringType(), True),
    StructField("budget", DoubleType(), True),
    StructField("homePage", StringType(), True),
    StructField("overview", StringType(), True),
    StructField("popularity", DoubleType(), True),
    StructField("yearReleaseDate", IntegerType(), True),
    StructField("releaseDate", DateType(), True),
    StructField("revenue", DoubleType(), True),
    StructField("durationTime", IntegerType(), True),
    StructField("movieStatus", StringType(), True),
    StructField("tagline", StringType(), True),
    StructField("voteAverage", DoubleType(), True),
    StructField("voteCount", IntegerType(), True)
])

In [0]:
movie_day1_df = spark.read \
                .option("header", True) \
                .schema(movie_schema) \
                .csv("/mnt/sacjccmoviehistory/bronze/2024-12-30/movie.csv") \
                .filter("yearReleaseDate < 2000") \
                .select("movieId","title","yearReleaseDate","releaseDate","durationTime")

In [0]:
movie_day1_df.createOrReplaceTempView("movies_day1")

In [0]:
from pyspark.sql.functions import upper;

movie_day2_df = spark.read \
                .option("header", True) \
                .schema(movie_schema) \
                .csv("/mnt/sacjccmoviehistory/bronze/2024-12-30/movie.csv") \
                .filter("yearReleaseDate BETWEEN 1998 AND 2005") \
                .select("movieId",upper("title").alias("title"),"yearReleaseDate","releaseDate","durationTime")

In [0]:
movie_day2_df.createOrReplaceTempView("movies_day2")

In [0]:
from pyspark.sql.functions import upper;

movie_day3_df = spark.read \
                .option("header", True) \
                .schema(movie_schema) \
                .csv("/mnt/sacjccmoviehistory/bronze/2024-12-30/movie.csv") \
                .filter("yearReleaseDate BETWEEN 1983 AND 1998 OR yearReleaseDate BETWEEN 2006 AND 2010") \
                .select("movieId",upper("title").alias("title"),"yearReleaseDate","releaseDate","durationTime")

- Crear una tabla Delta para almacenar los datos

In [0]:
%sql
CREATE TABLE IF NOT EXISTS movie_demo.movies_merge(
  movieId INT,
  title STRING,
  yearReleaseDate INT,
  releaseDate DATE,
  durationTime INT,
  createdDate DATE,
  updateDate DATE
)

#### Día 1

In [0]:
%sql
MERGE INTO movie_demo.movies_merge AS tgt --tabla destino
USING movies_day1 AS src-- tabla origen (Vista)
ON tgt.movieId = src.movieId
WHEN MATCHED THEN
    UPDATE SET
    tgt.title = src.title,
    tgt.yearReleaseDate = src.yearReleaseDate,
    tgt.releaseDate = src.releaseDate,
    tgt.durationTime = src.durationTime,
    tgt.updateDate = current_timestamp
WHEN NOT MATCHED THEN
    INSERT (movieId, title, yearReleaseDate, releaseDate, durationTime, createdDate)
    VALUES (movieId, title, yearReleaseDate, releaseDate, durationTime, current_timestamp)


In [0]:
%sql
SELECT * FROM movie_demo.movies_merge;

#### Día 2

In [0]:
%sql
MERGE INTO movie_demo.movies_merge AS tgt --tabla destino
USING movies_day2 AS src-- tabla origen (Vista)
ON tgt.movieId = src.movieId
WHEN MATCHED THEN
    UPDATE SET
    tgt.title = src.title,
    tgt.yearReleaseDate = src.yearReleaseDate,
    tgt.releaseDate = src.releaseDate,
    tgt.durationTime = src.durationTime,
    tgt.updateDate = current_timestamp
WHEN NOT MATCHED THEN
    INSERT (movieId, title, yearReleaseDate, releaseDate, durationTime, createdDate)
    VALUES (movieId, title, yearReleaseDate, releaseDate, durationTime, current_timestamp)

In [0]:
%sql
SELECT * FROM movie_demo.movies_merge;

#### Día 3

1. Merge pySpark

In [0]:
from delta.tables import *

deltaTablePeople = DeltaTable.forPath(spark, "/mnt/sacjccmoviehistory/demo/movies_merge") # Tabla origen

deltaTablePeople.alias("tgt") \
   .merge(
       movie_day3_df.alias("src"),
       "tgt.movieId = src.movieId"
   ) \
  .whenMatchedUpdate(set =
    {
        "tgt.title": "src.title",
        "tgt.yearReleaseDate": "src.yearReleaseDate",
        "tgt.releaseDate": "src.releaseDate",
        "tgt.durationTime": "src.durationTime",
        "tgt.updateDate": "current_timestamp()"
    }) \
  .whenNotMatchedInsert(values = {
      "movieId": "movieId",
      "title":"title",
      "yearReleaseDate": "yearReleaseDate",
      "releaseDate": "releaseDate",
      "durationTime": "durationTime",
      "createdDate": "current_timestamp()",
  }) \
.execute()



In [0]:
%sql
SELECT * FROM movie_demo.movies_merge;

#### History, Time Travel y Vacuum

1. Historia y Control de Versiones
2. Viaje en el Tiempo
3. Vacio

In [0]:
%sql
DESC HISTORY movie_demo.movies_merge;

- Acceder a una version especifica de la tabla

In [0]:
%sql
SELECT * FROM movie_demo.movies_merge VERSION AS OF 2;

- Acceder a una version especifica por medio de la fecha (timestamp)

In [0]:
%sql
SELECT * FROM movie_demo.movies_merge TIMESTAMP AS OF '2025-03-28T16:04:43.000+00:00'

- Acceder a un punto de la historia y almacenarlo en un DataFrame


In [0]:
df = spark.read.format("delta").option("timestampAsOf", "2025-03-28T16:04:43.000+00:00").load("/mnt/sacjccmoviehistory/demo/movies_merge")

- Eliminar permanentemente registros.
- Databricks mantiene los registros de la historia por un periodo de 7 dias, por mas que intentemos eliminar el historial de registros no se puede.
- Se puede forzar la eliminacion con el RETAIN 

In [0]:
%sql
VACUUM movie_demo.movies_merge;

- Eliminar toda la historia correspondiente a la tabla movie_demo.movies_merge, con una retencion de 0 horas. Es decir, no vamos a tener ninguna historia

In [0]:
%sql
SET spark.databricks.delta.retentionDurationCheck.enabled = false;  -- Forzamos la eliminacion de la historia
VACUUM movie_demo.movies_merge RETAIN 0 HOURS;

In [0]:
%sql
SELECT * FROM movie_demo.movies_merge;

- Nota: Podemos listar los puntos de la historia, pero no podemos acceder a ellas (Hemos vaciado el Vacuum)

In [0]:
%sql
DESC HISTORY movie_demo.movies_merge;

- Eliminamos registros que pertenecen al 2004
- Por medio de la historia podriamos recuperar esos registros eliminados

In [0]:
%sql
DELETE FROM movie_demo.movies_merge
WHERE yearReleaseDate = 2004;

In [0]:
%sql
DESC HISTORY movie_demo.movies_merge;

In [0]:
%sql
SELECT * FROM movie_demo.movies_merge VERSION AS OF 9;

- Recuperar los registros eliminados, por medio de la sentencia merge

In [0]:
%sql
MERGE INTO movie_demo.movies_merge AS tgt
USING movie_demo.movies_merge VERSION AS OF 9 AS src
ON tgt.movieId = src.movieId
WHEN NOT MATCHED THEN -- cuando no se tenga coincidencia en los valores de estas columnas (tgt.movieId = src.movieId), insertamos todos los registros en la tabla de destino (tgt)
    INSERT *;

In [0]:
%sql SELECT *  FROM movie_demo.movies_merge WHERE yearReleaseDate = 2004; 

#### Transaction Log en Delta Lake

In [0]:
%sql
CREATE TABLE IF NOT EXISTS movie_demo.movies_log(
  movieId INT,
  title STRING,
  yearReleaseDate INT,
  releaseDate DATE,
  durationTime INT,
  createdDate DATE,
  updateDate DATE
)
USING DELTA;

In [0]:
%sql
DESC HISTORY movie_demo.movies_log;

In [0]:
%sql 
INSERT INTO movie_demo.movies_log
     SELECT * FROM movie_demo.movies_merge
WHERE movieId = 125537;

In [0]:
%sql 
SELECT * FROM movie_demo.movies_log;

In [0]:
%sql
DESC HISTORY movie_demo.movies_log;

In [0]:
%sql 
INSERT INTO movie_demo.movies_log
     SELECT * FROM movie_demo.movies_merge
WHERE movieId = 133575;

In [0]:
%sql
DESC HISTORY movie_demo.movies_log;

In [0]:
%sql
DELETE FROM movie_demo.movies_log
WHERE movieId = 125537;

In [0]:
%sql DESC HISTORY movie_demo.movies_log; 

In [0]:
list = [118452,124606,125052, 125123, 125263, 125537, 126141, 133575, 142132, 146269, 157185]
for movieId in list:
    spark.sql(f"""INSERT INTO movie_demo.movies_log
              SELECT * FROM movie_demo.movies_merge
              WHERE movieId = {movieId}""")

In [0]:
%sql DESC HISTORY movie_demo.movies_log; 

In [0]:
%sql
INSERT INTO movie_demo.movies_log
SELECT * FROM movie_demo.movies_merge;

In [0]:
%sql DESC HISTORY movie_demo.movies_log; 

- Nota: Databricks mantiene los registros de transacciones durante 30 días. Luego de eso procede a eliminar los registros de transacciones.

#### Convertir formato "Parquet" a "Delta"

In [0]:
%sql
CREATE TABLE IF NOT EXISTS movie_demo.movies_convert_to_delta(
  movieId INT,
  title STRING,
  yearReleaseDate INT,
  releaseDate DATE,
  durationTime INT,
  createdDate DATE,
  updateDate DATE
)
USING PARQUET;

In [0]:
%sql
INSERT INTO movie_demo.movies_convert_to_delta
SELECT * FROM movie_demo.movies_merge;

In [0]:
%sql
CONVERT TO DELTA movie_demo.movies_convert_to_delta;

- Convertir una ruta de archivo en formato Parquet a formato Delta

In [0]:
df = spark.table("movie_demo.movies_convert_to_delta")

In [0]:
# Escribir los datos del df en una carpeta en formato Parquet.
df.write.format("parquet").save("/mnt/sacjccmoviehistory/demo/movies_convert_to_delta_new")

In [0]:
%sql
--Convertir archivo parquet a Delta
CONVERT TO DELTA parquet.`/mnt/sacjccmoviehistory/demo/movies_convert_to_delta_new`;